In [ ]:
# !pip install transformers
# !pip install -U sentence-transformers
!pip install wordsegment
!pip install pyvi
!pip install nltk
!python -m spacy download en_core_web_trf
!pip install spacy-transformers 

In [ ]:
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')


In [3]:
from google.colab import drive
drive.mount('/content/drive')
# sau khi run xong bước này, vào biểu tượng file bên phải -> vào drive/MyDrive/Colab Notebooks, tạo folder tên dataset -> upload 2 file dataset vào thư mục này

Mounted at /content/drive


In [24]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf

# Spacy for english word segmentation
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import spacy_transformers
# Pyvi for vietnam word segmentation
from pyvi import ViTokenizer

# import pandas as pd
import os
import io
import string
import re

In [25]:
# !ls "/content/drive/MyDrive/Colab Notebooks/dataset/en_sents"
dataset_path = "/content/drive/MyDrive/Colab Notebooks/dataset/"
spacy_en = spacy.load('en_core_web_trf')

# eos = '<eos>'
# bos = '<bos>'

In [32]:
def tokenize_en(en_dtset):
  en_return = []
  for sent in en_dtset:
    en_return.append([tok.text for tok in spacy_en.tokenizer(sent)])

  # add begin and end to sentence
  # for i in range(len(en_return)):
  #   en_return[i].insert(0, bos)
  #   en_return[i].insert(len(en_return[i]), eos)
  return en_return

def tokenize_vi(vi_dtset):
  vi_return = []
  for sent in vi_dtset:
    vi_return.append(ViTokenizer.tokenize(sent).split())

  # add begin and end to sentence
  # for i in range(len(vi_return)):
  #   vi_return[i].insert(0, bos)
  #   vi_return[i].insert(len(vi_return[i]), eos)
  return vi_return

en_sentences = open(dataset_path + 'en_sents', 'r').read().splitlines()
en_lst = [line.lower() for line in en_sentences[:10000]]

vi_sentences = open(dataset_path + 'vi_sents', 'r').read().splitlines()
vi_lst = [line.lower() for line in vi_sentences[:10000]]

en_seg = tokenize_en(en_lst)
# print(en_seg)

vi_seg = tokenize_vi(vi_lst)
en_seg



[['please', 'put', 'the', 'dustpan', 'in', 'the', 'broom', 'closet'],
 ['be', 'quiet', 'for', 'a', 'moment', '.'],
 ['read', 'this'],
 ['tom',
  'persuaded',
  'the',
  'store',
  'manager',
  'to',
  'give',
  'him',
  'back',
  'his',
  'money',
  '.'],
 ['friendship', 'consists', 'of', 'mutual', 'understanding'],
 ['are', 'you', 'going', 'to', 'come', 'tomorrow', '?'],
 ['see', 'to', 'this', 'matter', 'right', 'away', ',', 'will', 'you', '?'],
 ['i', 'showed', 'my', 'friends', 'these', 'picture', 'postcards', '.'],
 ['mary', 'is', 'the', 'youngest', 'of', 'the', 'three', 'sisters'],
 ['he', 'has', 'two', 'aunts', 'on', 'his', 'mother', "'s", 'side', '.'],
 ['that', 'is', 'what', 'i', 'want', 'to', 'know'],
 ['onions', 'can', 'be', 'used', 'in', 'many', 'dishes', '.'],
 ['the', 'rumor', 'is', 'not', 'true', 'as', 'far', 'as', 'i', 'know'],
 ['i', 'told', 'tom', 'to', 'clean', 'his', 'room', '.'],
 ['has', 'anybody', 'seen', 'my', 'beer', 'mug', '?'],
 ['i', 'could', 'not', 'speak', '

In [28]:
# Word embedding
# from gensim.models import Word2Vec

# vi_model = Word2Vec(vi_seg, size=100, window=2, min_count=1, sample=0.0001, workers=4, sg=0, negative=10, cbow_mean=1, iter=5)

class Language():
    def __init__(self, lines):
        self.lines = lines
        self.word2id = {}
        self.id2word = {}
        self.vocab = set()
        self.max_len = 0
        self.min_len = 0
        self.vocab_size = 0
        self.init_language_params()

    def init_language_params(self):
        self.word2id['<pad>'] = 0
        for line in self.lines:
            self.vocab.update(line)
        for id, word in enumerate(self.vocab):
            self.word2id[word] = id + 1
        for word, id in self.word2id.items():
            self.id2word[id] = word
        
        self.max_len = max([len(line) for line in self.lines])
        self.min_len = min([len(line) for line in self.lines])
        self.vocab_size = len(self.vocab) + 1
    
    def sent_to_vec(self, sent):
      return np.array([self.word2id[word] for word in sent])


scr_en = Language(en_seg)
tar_vi = Language(vi_seg)

scr_vec = [scr_en.sent_to_vec(sent) for sent in scr_en.lines]
tar_vec = [tar_vi.sent_to_vec(sent) for sent in tar_vi.lines]

scr_tensor = tf.keras.preprocessing.sequence.pad_sequences(scr_vec, scr_en.max_len, padding='post')
tar_tensor = tf.keras.preprocessing.sequence.pad_sequences(tar_vec, tar_vi.max_len, padding='post')

tar_tensor

array([[ 802,  260, 2860, ...,    0,    0,    0],
       [1398, 3602, 3134, ...,    0,    0,    0],
       [ 275,  288,    0, ...,    0,    0,    0],
       ...,
       [1939, 1540, 3368, ...,    0,    0,    0],
       [3339, 1958,  658, ...,    0,    0,    0],
       [3794, 2795, 2746, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# pip install transformers && pip install torch

In [ ]:
# from transformers import DistilBertTokenizerFast, DistilBertModel

# tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
# tokens = tokenizer.encode('This is a input.', return_tensors='pt')
# print("These are tokens!", tokens)
# for token in tokens[0]:
#     print("This are decoded tokens!", tokenizer.decode([token]))

In [ ]:
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# print(model.embeddings.word_embeddings(tokens))
# for e in model.embeddings.word_embeddings(tokens)[0]:
#     print("This is an embedding!", e)

In [ ]:
# from sentence_transformers import SentenceTransformer, models

# ## Step 1: use an existing language model
# word_embedding_model = models.Transformer('distilroberta-base')

# ## Step 2: use a pool function over the token embeddings
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# # print('pool: ', pooling_model)

# ## Join steps 1 and 2 using the modules argument
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# print('model: ', model)


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model:  SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)
